<a href="https://colab.research.google.com/github/lerouxl/Wandb_files_cleaning/blob/main/Clean_WandB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clean WandB

This code will remove unused artifact from WandB, feering some place.

In [ ]:
!pip install wandb

In [6]:
import wandb
from tqdm.notebook import tqdm
import time

To customise:

In [11]:
# Neural network files that do not have the epoch number (to skip)
# UPDATE HERE to match your files name convension
SKIP_FILES = ["latest", "best"]

# Extract epoch number.
# UPDATE HERE to match your files name convension
# My naming convesion is X_net.pth, so I only grab what is before the _
def extrat_epoch_number(name:str) -> str :
  "Extract the part of name containing the number of the epoch. MUST return an str"
  return name.split("_")[0]

# UPDATE HERE to add your project name
# You can see all your project name by runing the next cell
PROJECT_NAME = "project_name"

# UDPATE HERE to specify the extension of your model files
FILE_EXTENSION = ".pth"

Create the function that will remove files

In [3]:
def clean_model_weight_files(file: wandb.apis.public.File, every_x_epochs: int = 5, dry_run : bool = True, verbos : bool = True) -> None:
  """
  Remove saved model weights from the Wandb storage API.
  Will ignore file starting with "best" or "lattest".
  Args:
    - file: an wandb API file. This file will be removed if it was not created in the n * every_x_epochs (with n an int). 
            File name have to have the epoch number
    - every_x_epochs : int. Every x epochs, the model weights will be keep. Other weight will be removed.
    - dry_run: bool. If True, only print witch filles will be removed.
    - verbos: bool. If True print a message at every files removed.
  """
  file_name = file.name

  # Extract epoch number.
  # My naming convesion is X_net.pth, so I only grab what is before the _
  epoch_number = extrat_epoch_number(file_name)
  
  # Check if we are on the "best" or "latest" epochs 
  if epoch_number in SKIP_FILES:
    # If so, we quit
    return 

  epoch_number = int(epoch_number)
  to_del = bool(epoch_number % every_x_epochs)

  if to_del:
    message = f"Removing {file_name} from wandb"

    if dry_run:
      message = "DRY RUN " + message
    else:
      file.delete()
    
    if verbos:
      print(message)

Select the project then, for each runs, remove all saved model execept one every 5 epochs.

In [ ]:
# When using artifact api methods that don't have an entity or project
#  argument, you must provide that information when instantiating the wandb.Api

# You can see all your project name by runing the next cell
api = wandb.Api(overrides={"project": PROJECT_NAME})

for run in tqdm(api.runs(), desc= "Run progression"):
  for file in tqdm(run.files(),leave=False, desc = f"Files progression for {run.name}"):
    if file.name.endswith(FILE_EXTENSION):
      clean_model_weight_files(file = file,every_x_epochs= 5, dry_run = True, verbos = True)
    time.sleep(0.2) # Let's not be ban by Wandb API ^^

List all runs available

In [ ]:
for i in api.runs():
  print(i.name)

List all files

In [ ]:
for i in api.runs()[0].files():
  print(i.name)